# Description
This notebook fits the cultivation model by maximum-likelihood.

It does not create plots, but saves the MLE as a JSON file to the `/processed` directory.

In [1]:
import json
import numpy
import pandas
import pathlib
import scipy.optimize
import sys

import calibr8
import murefi

import models

DP_PROCESSED = pathlib.Path("processed")

In [2]:
cm_biomass = models.get_biomass_model()
cm_glucose = models.get_glucose_model()
model = models.MonodModel()
dataset = murefi.load_dataset(DP_PROCESSED / "cultivation_dataset.h5")
theta_mapping = models.get_parameter_mapping()
theta_mapping.as_dataframe()

,S0,X0,mu_max,K_S,Y_XS
rid,,,,,
A02,S0,X0_A02,mu_max,0.02,Y_XS
A03,S0,X0_A03,mu_max,0.02,Y_XS
A04,S0,X0_A04,mu_max,0.02,Y_XS
A05,S0,X0_A05,mu_max,0.02,Y_XS
A06,S0,X0_A06,mu_max,0.02,Y_XS
A07,S0,X0_A07,mu_max,0.02,Y_XS
A08,S0,X0_A08,mu_max,0.02,Y_XS
B02,S0,X0_B02,mu_max,0.02,Y_XS
B03,S0,X0_B03,mu_max,0.02,Y_XS


### Fit the model by MLE

In [3]:
# Create the objective function
objective = murefi.objectives.for_dataset(
    dataset=dataset,
    model=model,
    parameter_mapping=theta_mapping,
    calibration_models=[cm_glucose, cm_biomass],
)

# If the guess has a NaN likelihood there's a problem.
ll_guess = objective(theta_mapping.guesses)
if not numpy.isfinite(ll_guess):
    raise ValueError(f"Your guess is shit. objective(guess)={ll_guess}")

# Find the maximum with scipy
fit_result = scipy.optimize.minimize(
    objective,
    x0=theta_mapping.guesses,
    bounds=theta_mapping.bounds,
    # The callback gives us a bit of a progress bar...
    callback=lambda x: sys.stdout.write("."),
)
if calibr8.optimization._warn_hit_bounds(
    fit_result.x, theta_mapping.bounds, theta_mapping.theta_names
):
    display(fit_result)
print()
for tf, tname in zip(fit_result.x, theta_mapping.theta_names):
    print(f"{tname: <10}{tf}")
print()
print(fit_result)

..............................................................................................................................................................................................................................................................................................................................................................................................................................
S0        16.921204160580245
X0_A02    0.23079702377860556
X0_A03    0.3353940899191831
X0_A04    0.30114059900996454
X0_A05    0.2668014878312081
X0_A06    0.2495689835785395
X0_A07    0.2524224867487302
X0_A08    0.2411009888858393
X0_B02    0.29698851023110645
X0_B03    0.3564511698569336
X0_B04    0.2909474895883859
X0_B05    0.2552676170330398
X0_B06    0.2519720618709851
X0_B07    0.25585045777498977
X0_B08    0.23977281681218005
X0_C02    0.41572153692890357
X0_C03    0.3139246074529877
X0_C04    0.2731419939437678
X0_C05    0.25074836501513886
X0_C06    0.257421754152422

In [4]:
with open(DP_PROCESSED / "full_dataset_mle.json", "w") as jfile:
    json.dump(
        {k: v for k, v in zip(theta_mapping.parameters.keys(), fit_result.x)},
        jfile,
        indent=4,
    )

In [5]:
%load_ext watermark
%watermark

Last updated: 2021-05-06T12:07:12.242657+02:00

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.19.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 6
Architecture: 64bit

